<a href="https://colab.research.google.com/github/MowsamR/Sign-Language/blob/main/Sign_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python mediapipe matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 29.6 MB/s eta 0:00:00


In [2]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf


In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilitie

In [4]:
def mediapipe_detection(image, model):
	image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
	image.flags.writeable = False
	results = model.process(image)
	image.flags.writeable = True
	image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
	return image, results

In [5]:
def draw_landmarks(image, results):
	mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
	mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
	mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
	mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [6]:
def extract_keypoints(results):
	pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
	face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
	left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
	left_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
	return np.concatenate([pose, face, left_hand, left_hand])


In [11]:
import cv2
from google.colab.patches import cv2_imshow
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
		samples_path = os.path.join('Sign', 'Clips')
		should_break = False
		for sample in os.listdir(samples_path):
			if should_break:
				break

			print(sample)
			clips_path = os.path.join(samples_path, sample) 
      #clips_path = os.path.join(samples_path, sample): This line creates
      #a string containing the path to the current file or directory by joining the 'Clips' directory
      #path with the name of the current file or directory.
			if not os.path.isdir(clips_path):
				continue

			for clip_number, clip in enumerate(os.listdir(clips_path)):
				if should_break:
					break

				file = os.path.join(clips_path, clip)
				if not (os.path.isfile(file) and file.endswith('.mp4')):
					continue

				cap = cv.VideoCapture(file)

				if not cap.isOpened(): 
					print("Cannot open Video feed")
					exit()
				frame_number = 0
				while True:
					
					# Capture frame-by-frame
					ret, frame = cap.read()
					    
					# if frame is read correctly ret is True
					if not ret:
						print("Can't receive frame (stream end?). Exiting ...")
						break

					# Make detections
					image, results = mediapipe_detection(frame, holistic)

					key_points = extract_keypoints(results)
					print(key_points)
					npy_path = os.path.join('Dataset', str(sample), str(clip_number), str(frame_number))
					np.save(npy_path, key_points)

					draw_landmarks(image, results)

					# Display the resulting frame 
					image = cv.flip(image, 1)
					cv2_imshow(image)


					if cv.waitKey(1) == ord('q'):
						should_break = True
						break

					frame_number += 1

				# When everything done, release the capture
				cap.release() 
				cv.destroyAllWindows()

FileNotFoundError: ignored

In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
!git clone https://github.com/MowsamR/Sign-Language.git


Cloning into 'Sign-Language'...
remote: Enumerating objects: 4668, done.
remote: Counting objects: 100% (4668/4668), done.
remote: Compressing objects: 100% (4640/4640), done.
remote: Total 4668 (delta 27), reused 4668 (delta 27), pack-reused 0
Receiving objects: 100% (4668/4668), 29.29 MiB | 24.31 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [23]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/content', 'Sign-Language', 'Extracted_Keypoints') 

# Actions that we try to detect
actions = np.array(['Good', 'Hello', 'Thank_you'])

# This is the number of videos per action 
no_sequences = 50

# Each video has 30 frames
sequence_length = 30

label_map = {label:num for num, label in enumerate(actions)}



In [24]:
#sequences contains 
sequences, labels = [], []
#Loop through each action
for action in actions:
  #Loop through each video (no_sequence = video)
    for sequence in range(1,no_sequences+1):
        window = []
        #Loop through each frame in each video (sequence_length = frame)
        for frame_num in range(sequence_length):
            #Load the data in "{}.npy".format(frame_num) into res variable.
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            #all frames collected will be stored in window list sequentially until (0-29)
            window.append(res)

        #once the window list is full (contains 0-29 .npy files), the loop will start again from no_sequence videos.
        #all the 0-29 .npy files stored in windows will then be stored in sequences lists in that order until (1-50)
        sequences.append(window)

        #label_map[action] gets the index for the current action.
        #this index will be appended to labels list
        labels.append(label_map[action])

In [27]:
np.array(sequences).shape
#150 videos (50 videos each x 3 actions = 150)
#30 frames per video
#1662 landmarks per frame

(150, 30, 1662)

In [28]:
np.array(labels).shape


(150,)